In [8]:
import asyncio
import aiohttp
import time

async def download_one(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            print('Read {} from {}'.format(resp.content_length, url))

async def download_all(sites):
    tasks = [asyncio.create_task(download_one(site)) for site in sites]
    await asyncio.gather(*tasks)


def main():
    sites = [
        'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143655',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143656',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143657',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143658',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143659',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143660',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143661',
		'http://www.dapenti.com/blog/readforwx.asp?name=xilei&id=143662'
    ]
    start_time = time.perf_counter()
    asyncio.run(download_all(sites))
    end_time = time.perf_counter()  
    print('Download {} sites in {} seconds'.format(len(sites), end_time - start_time))

if __name__ == '__main__':
    main()


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
asyncio.run()
# 相当于
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(coro)
finally:
    loop.close()


In [ ]:
# 遵循以下为代码的规范
if io_bound:
    if io_slow:
        print('Use Asyncio')
    else:
        print('Use multi-threading')
else if cpu_bound:
    print('Use multi-processing')
    

现在有这么一个需求：输入一个列表，对于列表中的每个元素，我想计算 0 到这个元素的所有整数的平方和。

In [49]:
import time
def cpu_bound(number):
    print(sum(i * i for i in range(number)))

def calculate_sums(numbers):
    for number in numbers:
        cpu_bound(number)

def main():
    start_time = time.perf_counter()
    numbers = [10000000 + x for x in range(20)]
    calculate_sums(numbers)
    end_time = time.perf_counter()
    print('Calculation takes {} seconds'.format(end_time - start_time))

if __name__ == '__main__':
    main()

333333283333335000000
333333383333335000000
333333483333355000001
333333583333395000005
333333683333455000014
333333783333535000030
333333883333635000055
333333983333755000091
333334083333895000140
333334183334055000204
333334283334235000285
333334383334435000385
333334483334655000506
333334583334895000650
333334683335155000819
333334783335435001015
333334883335735001240
333334983336055001496
333335083336395001785
333335183336755002109
Calculation takes 16.193519082997227 seconds


In [52]:
# 并行版本
import time
import concurrent.futures

def calculate_sums_future(numbers):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(cpu_bound, numbers)

def calcuter_future():
    start_time = time.perf_counter()
    numbers = [10000000 + x for x in range(20)]
    calculate_sums_future(numbers)
    end_time = time.perf_counter()
    print('多进程版本, 耗时{} 秒'.format(end_time - start_time))

if __name__ == '__main__':
    calcuter_future()

333333283333335000000
333334183334055000204333333583333395000005
333333883333635000055333334283334235000285


333333983333755000091
333333483333355000001333333683333455000014333334383334435000385


333333783333535000030
333333383333335000000
333334083333895000140
333334883335735001240
333334483334655000506
333334983336055001496333335083336395001785

333334683335155000819
333334783335435001015333335183336755002109333334583334895000650


多进程版本, 耗时15.888232332996267 秒


In [46]:
# 动态规划版本
import time
import concurrent.futures
squ = {} # 用于存储中间结果

def cpu_dp(number):
	result = 0
	for i in range(number):
		if i not in squ.keys():
			squ[i] = i*i
		result += squ[i]
	print('number = {}, result = {}'.format(number, result))

def calculate_sums_dp(numbers):
	for number in numbers:
		cpu_dp(number)

def calcuter_dp(numbers):
	start_time = time.perf_counter()
	calculate_sums_dp(numbers)
	end_time = time.perf_counter()
	print('动态规划版本, 耗时{}秒'.format(end_time - start_time))

numbers = [10000000 + x for x in range(20)]
calcuter_dp(numbers)

number = 10000000, result = 333333283333335000000
number = 10000001, result = 333333383333335000000
number = 10000002, result = 333333483333355000001
number = 10000003, result = 333333583333395000005
number = 10000004, result = 333333683333455000014
number = 10000005, result = 333333783333535000030
number = 10000006, result = 333333883333635000055
number = 10000007, result = 333333983333755000091
number = 10000008, result = 333334083333895000140
number = 10000009, result = 333334183334055000204
number = 10000010, result = 333334283334235000285
number = 10000011, result = 333334383334435000385
number = 10000012, result = 333334483334655000506
number = 10000013, result = 333334583334895000650
number = 10000014, result = 333334683335155000819
number = 10000015, result = 333334783335435001015
number = 10000016, result = 333334883335735001240
number = 10000017, result = 333334983336055001496
number = 10000018, result = 333335083336395001785
number = 10000019, result = 333335183336755002109


In [48]:
# multiprocessing版本
import time
import concurrent.futures
import multiprocessing

def calculate_sums_multiprocessing(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)

def calcuter_multiprocessing(numbers):
    start_time = time.perf_counter()
    calculate_sums_multiprocessing(numbers)
    end_time = time.perf_counter()
    print('multiprocessing 版本, 耗时{}秒'.format(end_time-start_time))

numbers = [10000000 + x for x in range(20)]
calcuter_multiprocessing(numbers)


Process SpawnPoolWorker-29:
Process SpawnPoolWorker-31:
Process SpawnPoolWorker-30:
Traceback (most recent call last):
  File "/Users/du/miniforge3/envs/tf/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/du/miniforge3/envs/tf/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/du/miniforge3/envs/tf/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/du/miniforge3/envs/tf/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'cpu_bound' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/du/miniforge3/envs/tf/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/du/miniforge3/envs/tf/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._

KeyboardInterrupt: 